In [1]:
import numpy as np
import plotly.express as ex
import pandas as pd

# Singer Acceleration Model

Singer target maneuver model takes the target states to be position, velocity, and acceleration and assumes the target acceleration to be a first-order Markov process of the form:

$$
  a(k+1) = \rho_m a(k) + \sqrt{1 - \rho_m^2} \sigma_m r(k)
$$

where $\rho_m = e^{-\beta T}$, $\beta = 1/\tau_m$, $\tau_m, \sigma_m$ are target maneuver time constant and standard deviation and $r(k)$ is zero-mean, unit-standard deviation Gaussian distributed random variable. This can be collapsed to:

$$
  a(k+1) = e^{-\frac{T}{\tau_m}} a(k) + \sqrt{1 - e^{-\frac{T}{\tau_m}}} \sigma_m r(k)
$$

In [65]:
def acceleration(tau, sigma, T, n=100, rnd=None):
    normal = np.random.normal if rnd is None else rnd.normal
    rho = np.exp(-T / tau)
    res = []
    
    a_k = 0
    for i in range(n):
        a_k_1 = rho * a_k + np.sqrt(1 - rho*rho) * normal(0, sigma)
        res.append(a_k_1)
        a_k = a_k_1

    return np.array(res)

def calculate_dimension(time, a):
    v_last = 0; v = []
    p_last = 0; p = []

    time = [0] + np.diff(time).tolist()
    
    for dt, a_last in zip(time, a):
        p_last += v_last * dt + a_last*dt*dt/2
        v_last += dt*a_last
        p.append(p_last)
        v.append(v_last)
    
    return np.array((np.array(p), np.array(v), a)).T

def sample_singer(tau: float, sigma: float, T: float, n=100, seed=None):
    rnd = np.random.default_rng(seed=seed)
    time = np.arange(n) * T

    # generate position, velocity and acceleration independently in each dimension
    dims = [calculate_dimension(time, acceleration(tau, sigma, T, n, rnd)) for i in range(3)]

    # reshuffle to keep positions, velocities, and accelerations grouped
    ret = [dims[0][:,0], dims[1][:,0], dims[2][:,0],
           dims[0][:,1], dims[1][:,1], dims[2][:,1],
           dims[0][:,2], dims[1][:,2], dims[2][:,2]]
    
    return np.array(ret).T

# Sample traces

In [68]:
states = sample_singer(10, 1, 1, seed=1)

fig = ex.scatter_3d(x=states[:,0], y=states[:,1], z=states[:,2])
fig.update_traces(marker_size = 1)
fig

In [72]:
states = sample_singer(10, 1, 1)

fig = ex.scatter_3d(x=states[:,0], y=states[:,1], z=states[:,2])
fig.update_traces(marker_size = 1)
fig

In [73]:
states = sample_singer(10, 1, 1)

fig = ex.scatter_3d(x=states[:,0], y=states[:,1], z=states[:,2])
fig.update_traces(marker_size = 1)
fig